# The Battle of Neighborhoods- London Housing Market 2019 (Jan-May)
##### By Elva Ma

## 1. Background description 

Home price in London growth has been gradually slowing ever since the summer of 2016. A moribund market is preventing potential sellers from bringing their homes to the market and an ongoing decline in new instructions being listed for sale has intensified of late. A recent survey of U.K. residential property from the Royal Institution of Chartered Surveyors (RICS) concluded that Brexit is currently the main obstacle for market activity. And uncertainty over how the U.K. leaves the European Union was holding back both buyers and sellers of property. House prices may be continuing to fall with the number of new inquiries down for the eighth consecutive month, according to an industry survey.

## 2. Business Problem

Suppose I am a professional real estate trading consultant, my clients come to my office and ask me how can I assist them to purchase a suitable real estate in London in this uncertain economic? What they can put their money on? 

## 3. Data Collection

Price Paid Data includes information on all property sales in England and Wales that are sold for full market value and are lodged with registration. The files include standard and additional price paid data transactions received at HM Land Registry in the period from January 1st to May 31st in 2019 at https://www.gov.uk/government/statistical-data-sets/price-paid-data-downloads. The following fields comprise the address data included in Price Paid Data: Postcode; PAON Primary Addressable Object Name. Typically the house number or name; SAON Secondary Addressable Object Name. If there is a sub-building, for example, the building is divided into flats, there will be a SAON; Street; Locality; Town/City; District; County.

To explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we will access data through FourSquare API interface and arrange them as a data frame for visualization. By merging data on London properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we will be able to recommend profitable real estate investments.

In [1]:
import os # Operating System
import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium #import folium # map rendering library

import datetime as DT
import hmac
from geopy.geocoders import Nominatim
from geopy.distance import vincenty
# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         560 KB

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-

In [2]:
#Read the data from https://www.gov.uk/government/statistical-data-sets
df_london = pd.read_csv("http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2019.csv")

In [3]:
df_london.head()

,{87E1551E-C273-6405-E053-6C04A8C0B2EE},170000,2019-03-25 00:00,SN3 2QX,T,N,F,44,Unnamed: 8,KINGSWOOD AVENUE,PARK NORTH,SWINDON,SWINDON.1,SWINDON.2,A,A.1
0,{87E1551E-C274-6405-E053-6C04A8C0B2EE},119000,2019-04-12 00:00,SN2 2HZ,F,N,L,53,NaN,ROSE STREET,NaN,SWINDON,SWINDON,SWINDON,A,A
1,{87E1551E-C275-6405-E053-6C04A8C0B2EE},209000,2019-04-05 00:00,SN5 5GP,T,N,F,31,NaN,DANESTONE CLOSE,MIDDLELEAZE,SWINDON,SWINDON,SWINDON,A,A
2,{87E1551E-C276-6405-E053-6C04A8C0B2EE},213000,2019-03-29 00:00,SN5 4AZ,T,N,F,72,NaN,REIDS PIECE,PURTON,SWINDON,WILTSHIRE,WILTSHIRE,A,A
3,{87E1551E-C277-6405-E053-6C04A8C0B2EE},250000,2019-04-03 00:00,SN5 5FP,D,N,F,5,NaN,NEVIS CLOSE,SPARCELLS,SWINDON,SWINDON,SWINDON,A,A
4,{87E1551E-C278-6405-E053-6C04A8C0B2EE},198000,2019-04-12 00:00,SN3 6JP,T,N,F,34,NaN,BOLDREWOOD,NaN,SWINDON,SWINDON,SWINDON,A,A


## 4. Methodology 

Methodology section
The Methodology section will describe the main components of our analysis and predication system. The Methodology section comprises four stages:

### Explore and understand data

In [5]:
df_london.shape

(211899, 16)

This dataset consists of over 211899 rows and 16 columns. We will now prepare and preprocess data accordingly.

In [6]:
# create headers list

headers =['TUID','Sold_Price','Date','Postcode','Prop_Type','Old_New','Duration','PAON','SAON','Street','Locality_Name','Town_City','District','County','PPD_Cat_Type','Record_Status']

print("headers\n", headers)

headers
 ['TUID', 'Sold_Price', 'Date', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', 'SAON', 'Street', 'Locality_Name', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']


In [7]:
df_london.columns = headers
df_london.head()

,TUID,Sold_Price,Date,Postcode,Prop_Type,Old_New,Duration,PAON,SAON,Street,Locality_Name,Town_City,District,County,PPD_Cat_Type,Record_Status
0,{87E1551E-C274-6405-E053-6C04A8C0B2EE},119000,2019-04-12 00:00,SN2 2HZ,F,N,L,53,NaN,ROSE STREET,NaN,SWINDON,SWINDON,SWINDON,A,A
1,{87E1551E-C275-6405-E053-6C04A8C0B2EE},209000,2019-04-05 00:00,SN5 5GP,T,N,F,31,NaN,DANESTONE CLOSE,MIDDLELEAZE,SWINDON,SWINDON,SWINDON,A,A
2,{87E1551E-C276-6405-E053-6C04A8C0B2EE},213000,2019-03-29 00:00,SN5 4AZ,T,N,F,72,NaN,REIDS PIECE,PURTON,SWINDON,WILTSHIRE,WILTSHIRE,A,A
3,{87E1551E-C277-6405-E053-6C04A8C0B2EE},250000,2019-04-03 00:00,SN5 5FP,D,N,F,5,NaN,NEVIS CLOSE,SPARCELLS,SWINDON,SWINDON,SWINDON,A,A
4,{87E1551E-C278-6405-E053-6C04A8C0B2EE},198000,2019-04-12 00:00,SN3 6JP,T,N,F,34,NaN,BOLDREWOOD,NaN,SWINDON,SWINDON,SWINDON,A,A


In [8]:
# Format the date column
df_london['Date'] = df_london['Date'].apply(pd.to_datetime)

# Sort by Date of Sale
df_london.sort_values(by=['Date'],ascending=[False],inplace=True)

In [9]:
df_london_ppd = df_london.query("Town_City == 'LONDON'")

# Make a list of street names in LONDON
streets = df_london_ppd['Street'].unique().tolist()

In [10]:
df_grp_price = df_london_ppd.groupby(['Street'])['Sold_Price'].mean().reset_index()

# Give meaningful names to the columns
df_grp_price.columns = ['Street', 'Avg_Price']

In [11]:
#Input your Budget's Upper Limit and Lower Limit - Find the locations df_grp_price which fits your budget
df_affordable = df_grp_price.query("(Avg_Price >= 1000000) & (Avg_Price <= 2800000)")
df_affordable.head()

,Street,Avg_Price
1,ABBEY GARDENS,1557600.0
6,ABBOTS GARDENS,1310000.0
8,ABBOTSBURY CLOSE,1743000.0
19,ABERDEEN PARK,2100000.0
20,ABERDEEN PLACE,1700000.0


In [12]:
geolocator = Nominatim()

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  if __name__ == '__main__':


In [ ]:
df_affordable['city_coord'] = df_affordable['Street'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

In [ ]:
df_affordable[['Latitude', 'Longitude']] = df_affordable['city_coord'].apply(pd.Series)
df_affordable.head()

In [143]:
df_final=df_affordable.drop(columns=['city_coord'])
df_final.head()

,Street,Avg_Price,Latitude,Longitude
19,ABERDEEN PARK,2100000.0,35.928397,-86.519993
74,AIREDALE AVENUE,2025000.0,51.491383,-0.248097
87,ALBERT EMBANKMENT,2020000.0,51.493949,-0.121144
92,ALBION DRIVE,2180000.0,52.653656,1.312262
96,ALBION SQUARE,2450000.0,-41.273758,173.289393


In [144]:
address = 'London, UK'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


The geograpical coordinate of London City are 51.5073219, -0.1276474.


In [145]:
# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, price, street in zip(df_final['Latitude'], df_final['Longitude'], df_final['Avg_Price'], df_final['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='orange',
        fill=True,
        fill_color='red',
        fill_opacity=1,
        parse_html=False).add_to(map_london)  
    
map_london

### Define Foursquare Credentials and Version

In [146]:
# @hidden_cell
CLIENT_ID = 'OZ4SA1040WL2LH1KGQG1X1ZXWFR5DLWOQH0V33IRD1VQNK21' # Foursquare ID
CLIENT_SECRET = 'UQUZ1VNKTLHTJSUQZF44G2EX3QYMIQIXWLC4FGRSOS4ESD3B' # Foursquare Secret
VERSION = '20181206' # Foursquare API version

### Let's explore the first neighborhood in our dataframe
#### Create a function to process to all the streets in London

In [147]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *london_venues*

In [148]:
london_venues = getNearbyVenues(names=df_final['Street'],
                                   latitudes=df_final['Latitude'],
                                   longitudes=df_final['Longitude']
                                  )

ABERDEEN PARK
AIREDALE AVENUE
ALBERT EMBANKMENT
ALBION DRIVE
ALBION SQUARE
ALDERSGATE STREET
ASHCHURCH GROVE
ASHCHURCH PARK VILLAS
BEAR LANE
BEDFORD ROW
BEECHWOOD AVENUE
BOLSOVER STREET
BOROUGH HIGH STREET
BRAMPTON GROVE
BROMPTON ROAD
BRYANSTON SQUARE
BUCKINGHAM GATE
CADOGAN GARDENS
CADOGAN SQUARE
CAMDEN HIGH STREET
CAMDEN SQUARE
CAMPDEN HILL ROAD
CAMPDEN STREET
CANFIELD GARDENS
CANONBURY PARK SOUTH
CHALCOT ROAD
CHEPSTOW PLACE
CHEVENING ROAD
CHEYNE COURT
CLEVELAND SQUARE
CLIFTON STREET
CONIGER ROAD
CONYBEARE
CORK STREET
CREDITON HILL
CROFTDOWN ROAD
CURZON SQUARE
DARTMOUTH PARK ROAD
DAWSON PLACE
DEANSWAY
DEVEREUX LANE
DOVER PARK DRIVE
DUDLEY ROAD
EAST HEATH ROAD
EATON SQUARE
ENNISMORE GARDENS
ENNISMORE GARDENS MEWS
EVERSHED WALK
FAVART ROAD
FERNHURST ROAD
FLORAL STREET
FULHAM ROAD
GEORGE STREET
GOLBORNE ROAD
GORDON COTTAGES
GREEN LANES
GRESHAM WAY
HALSEY STREET
HARLEY GARDENS
HARROW ROAD
HARVIST ROAD
HAYDON WAY
HERONDALE AVENUE
HERTFORD AVENUE
HIGH CEDAR DRIVE
HILL STREET
HOLBEIN PLACE


KeyError: 'groups'

#### Let's check the size of the resulting dataframe

In [149]:
print(london_venues.shape)
london_venues.head()

(4528, 7)


,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,AIREDALE AVENUE,51.491383,-0.248097,Artisan,51.493530,-0.245071,Coffee Shop
1,AIREDALE AVENUE,51.491383,-0.248097,Angie's Little Food Shop,51.492860,-0.252044,Café
2,AIREDALE AVENUE,51.491383,-0.248097,Metropolis Studios,51.493881,-0.249818,Recording Studio
3,AIREDALE AVENUE,51.491383,-0.248097,Outsider Tart,51.492798,-0.251847,Bakery
4,AIREDALE AVENUE,51.491383,-0.248097,Kalamari,51.493507,-0.245302,Greek Restaurant


#### Let's check how many venues were returned for each neighborhood

In [150]:
london_venues.groupby('Street').count()

,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
AIREDALE AVENUE,40,40,40,40,40,40
ALBERT EMBANKMENT,51,51,51,51,51,51
ALBION DRIVE,5,5,5,5,5,5
ALBION SQUARE,28,28,28,28,28,28
ALDERSGATE STREET,73,73,73,73,73,73
ASHCHURCH GROVE,26,26,26,26,26,26
ASHCHURCH PARK VILLAS,27,27,27,27,27,27
BEECHWOOD AVENUE,2,2,2,2,2,2
BOLSOVER STREET,100,100,100,100,100,100


#### Let's find out how many unique categories can be curated from all the returned venues

In [151]:
# get the List of Unique Categories
print('There are {} uniques categories.'.format(len(london_venues['Venue Category'].unique())))

There are 330 uniques categories.


In [152]:
london_venues.shape

(4528, 7)

## Analyze Each Street

In [153]:
# one hot encoding
london_onehot = pd.get_dummies(london_venues[['Venue Category']], prefix="", prefix_sep="")

# add street column back to dataframe
london_onehot['Street'] = london_venues['Street'] 

# move street column to the first column
fixed_columns = [london_onehot.columns[-1]] + list(london_onehot.columns[:-1])

#fixed_columns
london_onehot = london_onehot[fixed_columns]

london_onehot.head()

,Street,Accessories Store,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Assisted Living,...,Watch Shop,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yakitori Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,AIREDALE AVENUE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AIREDALE AVENUE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AIREDALE AVENUE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,AIREDALE AVENUE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AIREDALE AVENUE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [168]:
london_grouped = london_onehot.groupby('Street').mean().reset_index()
london_grouped.head()

,Street,Accessories Store,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Assisted Living,...,Watch Shop,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yakitori Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,AIREDALE AVENUE,0.0,0.0,0.025,0.0,0.000000,0.025,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,ALBERT EMBANKMENT,0.0,0.0,0.000,0.0,0.039216,0.000,0.0,0.019608,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,ALBION DRIVE,0.0,0.0,0.000,0.0,0.000000,0.000,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,ALBION SQUARE,0.0,0.0,0.000,0.0,0.035714,0.000,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,ALDERSGATE STREET,0.0,0.0,0.000,0.0,0.041096,0.000,0.0,0.000000,0.0,...,0.0,0.0,0.013699,0.0,0.0,0.013699,0.0,0.0,0.0,0.0


In [156]:
london_grouped.shape

(124, 331)

### Print each neighborhood along with the top 5 most common venues

In [157]:
# What are the top 5 venues/facilities nearby profitable real estate investments?#

num_top_venues = 5

for hood in london_grouped['Street']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----AIREDALE AVENUE----
                 venue  freq
0                  Pub  0.12
1          Pizza Place  0.05
2   Italian Restaurant  0.05
3                 Café  0.05
4  Japanese Restaurant  0.05


----ALBERT EMBANKMENT----
         venue  freq
0         Park  0.10
1         Café  0.10
2          Pub  0.08
3        Hotel  0.08
4  Coffee Shop  0.06


----ALBION DRIVE----
                  venue  freq
0             Rock Club   0.4
1     Indian Restaurant   0.2
2  Fast Food Restaurant   0.2
3           Supermarket   0.2
4     Accessories Store   0.0


----ALBION SQUARE----
               venue  freq
0               Café  0.18
1        Coffee Shop  0.07
2         Restaurant  0.07
3  Indian Restaurant  0.07
4                Bar  0.07


----ALDERSGATE STREET----
                  venue  freq
0           Coffee Shop  0.08
1    Italian Restaurant  0.08
2                 Hotel  0.05
3     French Restaurant  0.05
4  Gym / Fitness Center  0.05


----ASHCHURCH GROVE----
                       ve

           venue  freq
0    Coffee Shop  0.17
1  Grocery Store  0.06
2            Pub  0.06
3         Bakery  0.04
4           Café  0.04


----FERNHURST ROAD----
               venue  freq
0               Lake   0.2
1        Supermarket   0.2
2  Convenience Store   0.2
3             Forest   0.2
4               Park   0.2


----FLORAL STREET----
            venue  freq
0         Theater  0.08
1  Clothing Store  0.05
2     Coffee Shop  0.05
3  Ice Cream Shop  0.05
4    Dessert Shop  0.03


----FULHAM ROAD----
                           venue  freq
0  Paper / Office Supplies Store   1.0
1             Photography Studio   0.0
2                      Pet Store   0.0
3            Peruvian Restaurant   0.0
4             Persian Restaurant   0.0


----GEORGE STREET----
              venue  freq
0       Coffee Shop  0.21
1    Clothing Store  0.10
2         Bookstore  0.06
3  Asian Restaurant  0.04
4          Platform  0.04


----GOLBORNE ROAD----
               venue  freq
0               Café

         venue  freq
0          Pub  0.11
1  Coffee Shop  0.07
2         Café  0.06
3        Hotel  0.05
4  Pizza Place  0.04


----OBSERVATORY GARDENS----
            venue  freq
0             Pub  0.05
1       Juice Bar  0.04
2      Restaurant  0.04
3  Clothing Store  0.04
4            Café  0.04


----ONSLOW CRESCENT----
            venue  freq
0             Pub  0.33
1             Bar  0.33
2  Cricket Ground  0.33
3     Pastry Shop  0.00
4        Pharmacy  0.00


----ORLANDO ROAD----
                 venue  freq
0    Mobile Phone Shop  0.17
1       Mattress Store  0.17
2     Department Store  0.17
3          Supermarket  0.17
4  Sporting Goods Shop  0.17


----ORMONDE PLACE----
                 venue  freq
0  American Restaurant  0.25
1       Cosmetics Shop  0.25
2     Tapas Restaurant  0.25
3           Restaurant  0.25
4     Recording Studio  0.00


----OSPRINGE ROAD----
            venue  freq
0             Pub  0.33
1  History Museum  0.17
2             Spa  0.17
3  Cricket Grou

## Define a function to return the most common venues/facilities nearby real estate investments

In [158]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Create the new dataframe and display the top 10 venues for each street

In [159]:
num_top_venues = 6

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Street'] = london_grouped['Street']

for ind in np.arange(london_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

venues_sorted.head()

,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
0,AIREDALE AVENUE,Pub,Italian Restaurant,Pizza Place,Hotel,Burger Joint,Thai Restaurant
1,ALBERT EMBANKMENT,Park,Café,Pub,Hotel,Coffee Shop,Restaurant
2,ALBION DRIVE,Rock Club,Supermarket,Indian Restaurant,Fast Food Restaurant,Zoo Exhibit,Food Stand
3,ALBION SQUARE,Café,Pub,Bar,Indian Restaurant,Restaurant,Coffee Shop
4,ALDERSGATE STREET,Italian Restaurant,Coffee Shop,Hotel,French Restaurant,Gym / Fitness Center,Plaza


In [160]:
venues_sorted.shape

(124, 7)

In [170]:
london_grouped.shape

(124, 331)

In [173]:
london_grouped=df_final

# Cluster Streets
##### Next we can start to cluster the properties by venues, facukutues, amenitites nearby the most profitable real estate investments in London.

In [213]:
#Distribute in 5 Clusters

# set number of clusters
kclusters = 5

london_grouped_clustering = london_grouped.drop('Street', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]

array([2, 2, 2, 0, 4, 0, 4, 1, 1, 4, 4, 2, 3, 3, 4, 0, 0, 0, 2, 1, 4, 3, 0,
       0, 4, 4, 4, 0, 3, 3, 0, 4, 2, 2, 0, 3, 3, 4, 2, 4, 0, 4, 0, 2, 1, 0,
       0, 0, 4, 0], dtype=int32)

In [214]:
#Dataframe to include Clusters

london_grouped_clustering=df_final
london_grouped_clustering.head()

,Street,Avg_Price,Latitude,Longitude,Cluster Labels
19,ABERDEEN PARK,2100000.0,35.928397,-86.519993,2
74,AIREDALE AVENUE,2025000.0,51.491383,-0.248097,2
87,ALBERT EMBANKMENT,2020000.0,51.493949,-0.121144,2
92,ALBION DRIVE,2180000.0,52.653656,1.312262,0
96,ALBION SQUARE,2450000.0,-41.273758,173.289393,4


In [215]:
london_grouped_clustering.shape

(134, 5)

In [216]:
df_final.shape

(134, 5)

In [217]:
london_grouped_clustering.dtypes

Street             object
Avg_Price         float64
Latitude          float64
Longitude         float64
Cluster Labels      int32
dtype: object

In [218]:
df_final.dtypes

Street             object
Avg_Price         float64
Latitude          float64
Longitude         float64
Cluster Labels      int32
dtype: object

In [219]:
# add clustering labels
london_grouped_clustering['Cluster Labels'] = kmeans.labels_

# merge london_grouped with london_data to add latitude/longitude for each neighborhood
london_grouped_clustering = london_grouped_clustering.join(venues_sorted.set_index('Street'), on='Street')

london_grouped_clustering.head() # check the last columns!

,Street,Avg_Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
19,ABERDEEN PARK,2100000.0,35.928397,-86.519993,2,NaN,NaN,NaN,NaN,NaN,NaN
74,AIREDALE AVENUE,2025000.0,51.491383,-0.248097,2,Pub,Italian Restaurant,Pizza Place,Hotel,Burger Joint,Thai Restaurant
87,ALBERT EMBANKMENT,2020000.0,51.493949,-0.121144,2,Park,Café,Pub,Hotel,Coffee Shop,Restaurant
92,ALBION DRIVE,2180000.0,52.653656,1.312262,0,Rock Club,Supermarket,Indian Restaurant,Fast Food Restaurant,Zoo Exhibit,Food Stand
96,ALBION SQUARE,2450000.0,-41.273758,173.289393,4,Café,Pub,Bar,Indian Restaurant,Restaurant,Coffee Shop


In [220]:
# Create Map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_grouped_clustering['Latitude'], london_grouped_clustering['Longitude'], london_grouped_clustering['Street'], london_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [222]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 0, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
92,2180000.0,Rock Club,Supermarket,Indian Restaurant,Fast Food Restaurant,Zoo Exhibit,Food Stand
105,2307473.0,Italian Restaurant,Coffee Shop,Hotel,French Restaurant,Gym / Fitness Center,Plaza
858,2200000.0,Hotel,Middle Eastern Restaurant,Italian Restaurant,Sandwich Place,Lebanese Restaurant,Coffee Shop
868,2240000.0,Hotel,Coffee Shop,Sandwich Place,Theater,Sushi Restaurant,Gym / Fitness Center
940,2310000.0,Liquor Store,Gastropub,Seafood Restaurant,Indian Restaurant,Food Stand,Fish & Chips Shop


In [223]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 1, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
255,2.600000e+06,Grocery Store,Pub,Coffee Shop,Park,Indian Restaurant,Japanese Restaurant
428,2.600000e+06,NaN,NaN,NaN,NaN,NaN,NaN
976,2.568800e+06,Pub,Coffee Shop,Pizza Place,Greek Restaurant,Supermarket,Music Venue
1919,2.535000e+06,Italian Restaurant,Restaurant,Café,Hotel,Pub,Coffee Shop
2345,2.623333e+06,Paper / Office Supplies Store,Zoo Exhibit,Film Studio,Fish Market,Flea Market,Food


In [224]:

london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 2, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
19,2100000.0,NaN,NaN,NaN,NaN,NaN,NaN
74,2025000.0,Pub,Italian Restaurant,Pizza Place,Hotel,Burger Joint,Thai Restaurant
87,2020000.0,Park,Café,Pub,Hotel,Coffee Shop,Restaurant
643,2094150.0,Coffee Shop,Restaurant,Café,Italian Restaurant,Pizza Place,Mediterranean Restaurant
942,2000000.0,Café,Italian Restaurant,Hotel,Boutique,Japanese Restaurant,Restaurant


In [226]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 3, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
652,2800000.0,Coffee Shop,Pub,Restaurant,Italian Restaurant,Food Truck,Hotel
713,2800000.0,Men's Store,Lake,Middle Eastern Restaurant,Zoo Exhibit,Food Stand,Fish & Chips Shop
988,2675000.0,Pub,Coffee Shop,Bakery,Yoga Studio,Indian Restaurant,Hotel
1218,2750000.0,Gastropub,Construction & Landscaping,Zoo Exhibit,Food Truck,Fish Market,Flea Market
1310,2800000.0,Bar,Lounge,Pizza Place,Gay Bar,Mexican Restaurant,Theater


In [227]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 4, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
96,2450000.0,Café,Pub,Bar,Indian Restaurant,Restaurant,Coffee Shop
254,2425000.0,Pub,Grocery Store,Coffee Shop,Indian Restaurant,Japanese Restaurant,Auto Garage
454,2450000.0,NaN,NaN,NaN,NaN,NaN,NaN
466,2500000.0,Restaurant,Performing Arts Venue,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market
808,2475000.0,Café,Italian Restaurant,Coffee Shop,Hotel,Exhibit,Boutique


## Conclusion

After processing all the above data and we can summaries and recommend that West London (Notting Hill, Kensington, Chelsea, Marylebone) and North-West London (Hampsted) might be considered highly profitable venues to purchase a real estate according to amenities and essential facilities surrounding. We also clustered 5 area that：
1.	Cluster 0: 1st most common venue in are club, restaurant hotel and liquor store, this cluster will be highly recommended for our client who want to invest for business traveler or sightseeing; 
2.	Cluster 1, 2 and 4: most common venue in those clusters are grocery stores, Pub, restaurants, cafe and supermarkets. Those area target home buyers prone to invest the residential live, people living in this area will be very convenient for shopping everything they need.
3.	Cluster 3: most common venue are coffee shop, men’s store, pub, and bar. This cluster will be called recreation area, it can be targeted for people who enjoy relaxation life after work.